In [2]:
import numpy as np
import functools as ft
import networkx as nx

In [3]:
def hdv(d):
    return np.random.choice([-1, 1], d)


def zero(d):
    return np.zeros((d,))


def bind(xs):
    return ft.reduce(lambda x, y: x * y, xs)


def bundle(xs):
    return ft.reduce(lambda x, y: x + y, xs)


def p(d):
    return np.random.shuffle(np.eye(d))


def inverse(P):
    return np.linalg.inv(P)


def permute(P, H):
    return P.dot(H)


def cosine_similarity(A, B):
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)

    if norm_A == 0 or norm_B == 0:
        return 0

    return np.dot(A, B) / (norm_A * norm_B)


def similarity(A, B):
    return np.dot(A, B) / len(A)


sim = cosine_similarity

In [4]:
class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def getVector(self, label):
        return self.vectors[label]

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: sim(V, x[1]))

In [5]:
def initVertices(graph, d=10000):
    for n in graph:
        graph.nodes[n]["hdv"] = hdv(d)

In [6]:
def initNodeMem(graph, memory):
    for n in graph:
        xs = map(lambda x: graph.nodes[x]["hdv"], list(graph.neighbors(n)))
        graph.nodes[n]["mem"] = bundle(xs)
        memory.addVector(f"mem{n}", graph.nodes[n]["mem"])

In [7]:
def retrain(graph, memory, threshold, iter=15):
    count = 0
    for i in range(iter):
        for n in graph:
            mem = graph.nodes[n]["mem"]
            finish = True
            for neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.neighbors(graph, n))
            ):
                if sim(mem, neighbor) < threshold:
                    mem = bundle([mem, neighbor])
                    finish = False
                    print("here")

            for non_neighbor in map(
                lambda x: graph.nodes[x]["hdv"], list(nx.non_neighbors(graph, n))
            ):
                if sim(mem, non_neighbor) > threshold:
                    mem = bundle([mem, -non_neighbor])
                    finish = False
                    print("here")

            if finish:
                return

            graph.nodes[n]["mem"] = mem
            memory.addVector(f"mem{n}_{i}", mem)

In [8]:
def initGraph(graph, memory):
    G = []
    for n in graph.nodes():
        G.append(bind([graph.nodes[n]["hdv"], graph.nodes[n]["mem"]]))
    G = bundle(G) / 2
    memory.addVector("G", G)
    return G

In [9]:
def checkEdge(G, A, B, threshold):
    return sim(B, bind([G, A])) > threshold

In [10]:
def nodeMemoryReconstruction(G, xs, iter=15):
    if iter == 0:
        return list(map(lambda H: bind([G, H]), xs))

    mems = nodeMemoryReconstruction(G, xs, iter - 1)

    newMems = []
    b = [bind(x) for x in zip(mems, xs)]
    for i in range(len(xs)):
        bmem = bundle(b[:i] + b[i + 1 :])
        newMems.append(bind([xs[i], bundle([G, -bmem])]))

    return newMems

In [11]:
def graphMemoryReconstruction(G, xs, iter=15):
    raise NotImplementedError
    if iter == 0:
        return (G, zeros(len(G)))

    (Gi, Ni) = graphMemoryReconstruction(G, xs, iter - 1)
    Gii = bundle([Gi, -Ni])

    for i in range(iter):
        print(i)

    return bundle([Gi, -edges])

In [28]:
# visited = dict()
visited = {}


def shortestPath(G, A, B, xs, distance, threshold, memory):
    label, value = memory.cleanup(bind([G, A]))

    if np.array_equiv(A, B):
        return 0, label

    if label in visited and visited[label][0] <= distance:
        return visited[label][1], visited[label][2]

    visited[label] = [distance, 99, "memnill"]

    neighbours = list(filter(lambda x: checkEdge(G, A, x, threshold), xs))

    if len(neighbours) == 0:
        return 99, "memnill"

    dis, lab = min(
        list(
            map(
                lambda a: shortestPath(G, a, B, xs, distance + 1, threshold, memory),
                neighbours,
            )
        ),
        key=lambda x: x[0],
    )
    visited[label][1] = dis + 1
    visited[label][2] = label + lab
    return dis + 1, label + lab

---


# Tests


In [13]:
def testCheckEdge(G, graph, threshold):
    count = 0

    for n in graph.nodes():
        for m in graph.nodes():
            if n == m:
                continue
            exist = graph.has_edge(n, m)
            check = checkEdge(
                G, graph.nodes[n]["hdv"], graph.nodes[m]["hdv"], threshold
            )
            if exist != check:
                count += 1
                print(n, m, exist, check)

    print(count, "%.5f" % round(count / EDGES, 5))

In [14]:
def testNodeMemoryReconstruction(G, graph, iter=15):
    for i in range(iter):
        memsi = nodeMemoryReconstruction(
            G, list(map(lambda x: graph.nodes[x]["hdv"], graph.nodes())), iter=i
        )
        print(
            f"{0}_{i:02} =>",
            "%.10f" % abs(sim(graph.nodes[0]["mem"], memsi[0])),
        )

---


# Main


In [15]:
NODES, EDGES = 30, 150
DIMENSIONS, THRESHOLD, ITER = 10000, 0.047, 4

In [16]:
def main():
    graph = nx.gnm_random_graph(NODES, EDGES)
    memory = ItemMemory()

    initVertices(graph, DIMENSIONS)
    initNodeMem(graph, memory)
    retrain(graph, memory, THRESHOLD, ITER)
    G = initGraph(graph, memory)
    print("G =>", G)
    # testNodeMemoryReconstruction(G, graph, 1)
    # testCheckEdge(G, graph, THRESHOLD)


# main()

In [17]:
# graph = nx.gnm_random_graph(NODES, EDGES)
# memory = ItemMemory()

# initVertices(graph, DIMENSIONS)
# initNodeMem(graph, memory)
# G = initGraph(graph, memory)
# print("G =>", G)

In [18]:
# testNodeMemoryReconstruction(G, graph, 15)

In [19]:
# sim(graph.nodes[0]["mem"], graph.nodes[0]["mem"])

---


In [20]:
mem = ItemMemory()

g = nx.gnm_random_graph(NODES, 45)
initVertices(g, DIMENSIONS)
initNodeMem(g, mem)
G = initGraph(g, mem)

for n in g.nodes():
    print(g.edges(n))

[(0, 13), (0, 24), (0, 27)]
[(1, 11), (1, 15)]
[(2, 27)]
[(3, 8)]
[(4, 26), (4, 18), (4, 12), (4, 11), (4, 25)]
[(5, 7), (5, 20), (5, 13), (5, 8)]
[(6, 23), (6, 21), (6, 27), (6, 22)]
[(7, 5), (7, 23)]
[(8, 3), (8, 5)]
[(9, 26), (9, 16), (9, 21), (9, 29), (9, 14)]
[(10, 27)]
[(11, 1), (11, 4), (11, 21)]
[(12, 4)]
[(13, 21), (13, 0), (13, 5)]
[(14, 9), (14, 23)]
[(15, 1), (15, 29)]
[(16, 9), (16, 24)]
[(17, 29)]
[(18, 26), (18, 4), (18, 23)]
[(19, 26), (19, 23), (19, 27), (19, 25)]
[(20, 23), (20, 5), (20, 21)]
[(21, 13), (21, 6), (21, 9), (21, 20), (21, 11)]
[(22, 23), (22, 6), (22, 28)]
[(23, 7), (23, 6), (23, 20), (23, 22), (23, 19), (23, 18), (23, 14)]
[(24, 16), (24, 0), (24, 27)]
[(25, 19), (25, 27), (25, 4)]
[(26, 9), (26, 18), (26, 19), (26, 4)]
[(27, 2), (27, 19), (27, 10), (27, 6), (27, 24), (27, 25), (27, 0)]
[(28, 22)]
[(29, 17), (29, 9), (29, 15)]


In [21]:
hdvs = list(map(lambda x: g.nodes[x]["hdv"], g.nodes()))
origin = 0

In [27]:
for n in g.nodes():
    print(
        f"{n:02} =>",
        "T" if mem.cleanup(bind([G, hdvs[n]]))[0] == f"mem{n}" else "F",
        mem.cleanup(bind([G, hdvs[n]]))[0],
    )

00 => T mem0
01 => T mem1
02 => T mem2
03 => T mem3
04 => T mem4
05 => T mem5
06 => T mem6
07 => T mem7
08 => T mem8
09 => T mem9
10 => F mem2
11 => T mem11
12 => T mem12
13 => T mem13
14 => T mem14
15 => T mem15
16 => T mem16
17 => T mem17
18 => T mem18
19 => T mem19
20 => T mem20
21 => T mem21
22 => T mem22
23 => T mem23
24 => T mem24
25 => T mem25
26 => T mem26
27 => T mem27
28 => T mem28
29 => T mem29


In [29]:
for m in g.nodes():
    if origin == m:
        continue
    path = nx.shortest_path(g, origin, m)
    oracle = len(path) - 1
    visited = {}
    test = shortestPath(G, hdvs[origin], hdvs[m], hdvs, 0, THRESHOLD, mem)
    testPath = list(map(int, test[1].split("mem")[1:]))
    print(
        f"{origin}_{m:02} =>",
        oracle,
        test[0],
        "T  " if oracle == test[0] else "F X",
        path,
        testPath,
        nx.is_path(g, testPath),
    )

0_01 => 4 4 T   [0, 13, 21, 11, 1] [0, 13, 21, 11, 1] True
0_02 => 2 2 T   [0, 27, 2] [0, 27, 2] True
0_03 => 4 4 T   [0, 13, 5, 8, 3] [0, 13, 5, 8, 3] True
0_04 => 3 3 T   [0, 27, 25, 4] [0, 27, 25, 4] True
0_05 => 2 2 T   [0, 13, 5] [0, 13, 5] True
0_06 => 2 2 T   [0, 27, 6] [0, 27, 6] True
0_07 => 3 3 T   [0, 13, 5, 7] [0, 13, 5, 7] True
0_08 => 3 3 T   [0, 13, 5, 8] [0, 13, 5, 8] True
0_09 => 3 3 T   [0, 13, 21, 9] [0, 13, 21, 9] True
0_10 => 2 2 T   [0, 27, 10] [0, 27, 2] True
0_11 => 3 3 T   [0, 13, 21, 11] [0, 13, 21, 11] True
0_12 => 4 4 T   [0, 27, 25, 4, 12] [0, 27, 25, 4, 12] True
0_13 => 1 1 T   [0, 13] [0, 13] True
0_14 => 4 4 T   [0, 13, 21, 9, 14] [0, 13, 21, 9, 14] True
0_15 => 5 5 T   [0, 13, 21, 11, 1, 15] [0, 13, 21, 9, 29, 15] True
0_16 => 2 2 T   [0, 24, 16] [0, 24, 16] True
0_17 => 5 5 T   [0, 13, 21, 9, 29, 17] [0, 13, 21, 9, 29, 17] True
0_18 => 4 4 T   [0, 27, 19, 26, 18] [0, 27, 6, 23, 18] True
0_19 => 2 2 T   [0, 27, 19] [0, 27, 19] True
0_20 => 3 3 T   [0, 1

In [24]:
testCheckEdge(G, g, THRESHOLD)

0 0.00000
